In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from llm import generate_responses, generate_prompt
from collections import defaultdict

def render_comparison(data):
    html = """
        <style>
            table { 
                border-collapse: collapse; 
                width: 100%; 
                font-family: sans-serif; 
                table-layout: auto; /* let columns auto-size */
            }
            th, td { 
                border: 1px solid #ccc; 
                padding: 8px; 
                text-align: left; 
                vertical-align: top; 
                /* remove fixed width here */
            }
            th { 
                background-color: #f2f2f2; 
            }
            td pre { 
                white-space: pre-wrap; 
                word-break: break-word; 
            }
        </style>
        <table>
            <tr><th>Problem ID</th><th>Original Plan</th><th>NL Description</th><th>Summary</th><th>Detail</th></tr>
        """
    for k, values in data.items():
        line = "<tr><td>" + str(k) + "</td>"
        for v in values:
            line += f"<td><pre>{v}</pre></td>"
        line += "</tr>"
        html += line
    html += "</table>"
    return html





In [ ]:
plan_file_path = "../../data/plans/plans.txt"
plans = []
with open(plan_file_path, 'r') as f:
    lines = f.readlines()
    count = 0
    for line in lines:
        if count >=10:
            break
        item = line.split("&&")
        domain = item[0].strip()
        prob = item[1].strip()
        plan = line.split("&&")[-1].strip()
        if domain != "transport":
            continue
        plans.append((prob, plan))
        count += 1

def process_plans(plans, prompt_name):
    print(prompt_name)
    with open(f"./nl/transport_{prompt_name}.txt", 'w') as f:
        for prob, plan in plans:
            print("Processing:", prob)
            prompt = generate_prompt(prompt_name, {'plan': plan})
            res = generate_responses('gpt-4o-mini', prompt)
            f.write(f"{prob}&&{plan}&&{res['content']}\n")
            f.flush()





In [5]:
process_plans(plans, "p2nl")


p2nl


FileNotFoundError: [Errno 2] No such file or directory: './nl/transport_p2nl.txt'

In [ ]:
process_plans(plans, "p2nl-summary")


In [ ]:
process_plans(plans, "p2nl-detail")

In [ ]:
data = defaultdict(list)
with open("./nl/transport_p2nl.txt", 'r') as f:
    for line in f:
        item = line.split("&&")
        prob = item[0].strip()
        plan = item[1].strip()
        nl = item[2].strip()
        data[prob] = [ plan, nl]
with open("./nl/transport_p2nl-summary.txt", 'r') as f:
    for line in f:
        item = line.split("&&")
        prob = item[0].strip()
        summary = item[2].strip()
        if prob in data:
            data[prob].append(summary)
with open("./nl/transport_p2nl-detail.txt", 'r') as f:
    for line in f:
        item = line.split("&&")
        prob = item[0].strip()
        detail = item[2].strip()
        if prob in data:
            data[prob].append(detail)

html = render_comparison(data)
with open("./views/p2nl.html", "w", encoding="utf-8") as f:
    f.write(html)